<a href="https://colab.research.google.com/github/thomd/stylegan2-ada-experiments/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

StyleGAN2 works only with TensorFlow1

In [1]:
%tensorflow_version 1.x
!nvidia-smi

TensorFlow 1.x selected.
Thu Dec  2 14:26:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                               

Mount your Drive to the Colab notebook

In [6]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
str(content_path)

'/content'

In [8]:
#stylegan2_repo_url = 'https://github.com/NVlabs/stylegan2-ada'
stylegan2_repo_url = 'https://github.com/dvschultz/stylegan2-ada'
project_path = drive_path / 'MyDrive' / 'StyleGAN2-ADA'
stylegan2_repo_path = project_path / 'stylegan2-ada'

if not project_path.is_dir():
    %mkdir "{project_path}"
%cd "{project_path}"

for dir in ['in', 'out', 'datasets', 'training']:
    if not (project_path / dir).is_dir():
        %mkdir {dir}

if not (project_path / 'datasets' / 'source').is_dir():
    %mkdir "{project_path / 'datasets' / 'source'}"

# Clone or Update StyleGAN2-ADA
if stylegan2_repo_path.is_dir():
    !git -C "{stylegan2_repo_path}" fetch origin
    !git -C "{stylegan2_repo_path}" checkout origin/main -- *.py
else:
    !git clone {stylegan2_repo_url}

/content/drive/MyDrive/StyleGAN2-ADA


In [13]:
local_dataset_path = drive_path / 'MyDrive' / 'Datasets'
if not local_dataset_path.is_dir():
    %mkdir "{local_dataset_path}"


## Data Upload

upload training dataset as zip file into `Datasets` folder on Google Drive

In [17]:
import zipfile

with zipfile.ZipFile(str(local_dataset_path / 'biked_small.zip'), 'r') as zip_ref:
    zip_ref.extractall(str(local_dataset_path))

convert image dataset to a `.tfrecords` format that StyleGAN2-ADA can read

In [12]:
stylegan2_repo_path

PosixPath('/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada')

In [18]:
local_images_path = local_dataset_path / 'biked_small'
local_tfr_images_path = local_dataset_path / 'biked_small_tfr'

if local_tfr_images_path.is_dir():
    print(f'Delete current dataset folder ({local_tfr_images_path}) to regenerate tfrecords')
else:
    %mkdir "{local_tfr_images_path}"
    !python "{stylegan2_repo_path / 'dataset_tool.py'}" create_from_images "{local_tfr_images_path}" "{local_images_path}"

Loading images from "/content/drive/MyDrive/Datasets/biked_small"
Creating dataset "/content/drive/MyDrive/Datasets/biked_small_tfr"
/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 300 images.


## Train the model

In [37]:
!python "{stylegan2_repo_path / 'train.py'}" --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional

In [40]:
training_path = project_path / 'training' / 'biked_small'
if not training_path.is_dir():
    %mkdir "{training_path}"

snapshot_count = 2          # how often should the model generate samples and a .pkl file?
mirrored = True             # should the images be mirrored left to right?
mirroredY = False           # should the images be mirrored top to bottom?
metric_list = None          # metrics? 
augs = 'bgc'                # which augments?

resume_from = 'ffhq1024'    # see https://github.com/NVlabs/ffhq-dataset

#!python "{stylegan2_repo_path / 'train.py'}" --outdir="{training_path}" \
#    --data="{local_tfr_images_path}" --resume="{resume_from}" \
#    --snap={snapshot_count} --augpipe={augs} \
#    --mirror={mirrored} --mirrory={mirroredY} \
#    --metrics={metric_list} --dry-run

!python "{stylegan2_repo_path / 'train.py'}" --outdir="{training_path}" \
    --data="{local_tfr_images_path}" \
    --snap={snapshot_count} --augpipe={augs} \
    --mirror={mirrored} --mirrory={mirroredY} \
    --metrics={metric_list} --kimg=1000

Traceback (most recent call last):
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/train.py", line 645, in <module>
    main()
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/train.py", line 637, in main
    run_training(**vars(args))
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/train.py", line 522, in run_training
    training_loop.training_loop(**training_options)
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/training/training_loop.py", line 252, in training_loop
    tflib.run([G_train_op, data_fetch_op])
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/dnnlib/tflib/tfutil.py", line 33, in run
    return tf.get_default_session().run(*args, **kwargs)
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py", line 956, in run
    run_metadata_ptr)
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py", line 1180, in _run
    feed_dict_tensor, options, run_metadata)
  File "/tensorflo

Training does periodically save the result and the model file `*.pkl` based on the provided `snapshot_count`. 

Once you think that the result is good enough or the FID starts to plateau, you can stop training and use the last saved `*.pkl` file.

In [26]:
%pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-images --help 

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2342912/45929032 bytes (5.1%)5660672/45929032 bytes (12.3%)8904704/45929032 bytes (19.4%)12124160/45929032 bytes (26.4%)15409152/45929032 bytes (33.5%)18456576/45929032 bytes (40.2%)21741568/45929032 bytes (47.3%)24846336/45929032 bytes (54.1%)28188672/45929032 bytes (61.4%)31268864/45929032 bytes (68.1%)34545664/45929032 bytes (75.2%)37789696/45929032 bytes (82.3%)

In [36]:
from numpy import random
seed_init = random.randint(10000)
nbr_images = 6

#generation_from = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/metfaces.pkl'
generation_from = training_path / '00000-biked_small_tfr-mirror-auto1-bgc-resumeffhq1024' / 'network-snapshot-000000.pkl'

!python "{stylegan2_repo_path / 'generate.py'}" generate-images \
    --outdir="{project_path / 'out'}" --trunc=0.7 \
    --seeds={seed_init}-{seed_init+nbr_images-1} \
    --network={str(generation_from)}

Loading networks from "/content/drive/MyDrive/StyleGAN2-ADA/training/biked_small/00000-biked_small_tfr-mirror-auto1-bgc-resumeffhq1024/network-snapshot-000000.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 5717 (0/6) ...
Generating image for seed 5718 (1/6) ...
Generating image for seed 5719 (2/6) ...
Generating image for seed 5720 (3/6) ...
Generating image for seed 5721 (4/6) ...
Generating image for seed 5722 (5/6) ...
